<a href="https://colab.research.google.com/github/imarariyal/DSPL-ICW/blob/main/DSPL_ICW_DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import random
import warnings
warnings.filterwarnings('ignore')

In [2]:
#read the csv
df = pd.read_csv('/content/indicators_lka.csv')
df.head()

,Country Name,Country ISO3,Year,Indicator Name,Indicator Code,Value
0,#country+name,#country+code,#date+year,#indicator+name,#indicator+code,#indicator+value+num
1,Sri Lanka,LKA,2022,Fertilizer consumption (% of fertilizer produc...,AG.CON.FERT.PT.ZS,1461.39775839426
2,Sri Lanka,LKA,2021,Fertilizer consumption (% of fertilizer produc...,AG.CON.FERT.PT.ZS,1461.39775839426
3,Sri Lanka,LKA,2020,Fertilizer consumption (% of fertilizer produc...,AG.CON.FERT.PT.ZS,2838.09421995635
4,Sri Lanka,LKA,2019,Fertilizer consumption (% of fertilizer produc...,AG.CON.FERT.PT.ZS,1803.55490605544


In [3]:
df.shape

(76315, 6)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76315 entries, 0 to 76314
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Country Name    76315 non-null  object
 1   Country ISO3    76315 non-null  object
 2   Year            76315 non-null  object
 3   Indicator Name  76315 non-null  object
 4   Indicator Code  76315 non-null  object
 5   Value           76315 non-null  object
dtypes: object(6)
memory usage: 3.5+ MB


In [5]:
df.describe()

,Country Name,Country ISO3,Year,Indicator Name,Indicator Code,Value
count,76315,76315,76315,76315,76315,76315
unique,2,2,65,3194,3195,46300
top,Sri Lanka,LKA,2016,Net migration,NV.AGR.TOTL.ZS,0
freq,76314,76314,3029,192,192,2737


In [6]:
#drop row as it is unnecessary
df.drop(index=0, inplace=True)

In [7]:
#removing columns that dont add valuable insights
df.drop(['Country Name','Country ISO3', 'Indicator Code'], axis=1, inplace=True)

In [8]:
#Rename columns for readability
df.rename(columns={'Indicator Name': 'Indicator'}, inplace=True)

In [9]:
df.head()

,Year,Indicator,Value
1,2022,Fertilizer consumption (% of fertilizer produc...,1461.39775839426
2,2021,Fertilizer consumption (% of fertilizer produc...,1461.39775839426
3,2020,Fertilizer consumption (% of fertilizer produc...,2838.09421995635
4,2019,Fertilizer consumption (% of fertilizer produc...,1803.55490605544
5,2018,Fertilizer consumption (% of fertilizer produc...,1312.52008353088


In [10]:
#check for null values
df.isnull().sum()

,0
Year,0
Indicator,0
Value,0


In [11]:
#checking for duplicates
duplicates = df[df.duplicated(subset=['Indicator', 'Year', 'Value'], keep=False)]
print(duplicates.sort_values(by=['Indicator', 'Year']))

       Year                                          Indicator  \
22976  2000            Access to electricity (% of population)   
24229  2000            Access to electricity (% of population)   
67086  2000            Access to electricity (% of population)   
22975  2001            Access to electricity (% of population)   
24228  2001            Access to electricity (% of population)   
...     ...                                                ...   
61386  2020  Women's share of population ages 15+ living wi...   
32733  2021  Women's share of population ages 15+ living wi...   
61385  2021  Women's share of population ages 15+ living wi...   
32732  2022  Women's share of population ages 15+ living wi...   
61384  2022  Women's share of population ages 15+ living wi...   

                  Value  
22976              70.3  
24229              70.3  
67086              70.3  
22975              63.6  
24228              63.6  
...                 ...  
61386  29.7619047619048  

In [12]:
#clean text fields properly again just to be safe
df['Indicator'] = df['Indicator'].str.strip()

#drop duplicates based ONLY on important columns
df = df.drop_duplicates(subset=['Indicator', 'Year', 'Value'])

#reset index after dropping
df = df.reset_index(drop=True)

#check again
print("Number of duplicates left:", df.duplicated(subset=['Indicator', 'Year', 'Value']).sum())

Number of duplicates left: 0


In [13]:
#data types
df.dtypes

,0
Year,object
Indicator,object
Value,object


In [14]:
#change the data types
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

In [15]:
#Keep data from 2000
df = df[df['Year'] >= 2000]

In [16]:
#clean inconsistent values in the Value column
df['Value'] = (
    df['Value']
    .astype(str)                     # ensure it's string
    .str.replace(',', '', regex=False)  # remove commas
    .str.replace('$', '', regex=False)  # remove $ if needed
    .str.strip()                     # remove whitespace
)

# Now convert to numeric
df['Value'] = pd.to_numeric(df['Value'], errors='coerce')

In [17]:
df.dropna(subset=['Year', 'Value'], inplace=True)

In [18]:
df.dtypes

,0
Year,int64
Indicator,object
Value,float64


In [19]:
#checking for unique values
df['Indicator'].unique()

array(['Fertilizer consumption (% of fertilizer production)',
       'Fertilizer consumption (kilograms per hectare of arable land)',
       'Agricultural land (sq. km)', ...,
       'Public and publicly guaranteed debt service (% of exports of goods, services and primary income)',
       'Multilateral debt service (TDS, current US$)',
       'Multilateral debt service (% of public and publicly guaranteed debt service)'],
      dtype=object)

In [20]:
#checking for unique values
df['Year'].unique()

array([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012,
       2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001,
       2000, 2023])

In [21]:
#checking for unique values
df['Value'].unique()

array([1461.39775839, 2838.09421996, 1803.55490606, ...,   19.09578728,
         16.9775547 ,   13.20911737])

In [24]:
#final cleaned df shape and overview
print(df.shape)
print(df.head())

(33320, 3)
   Year                                          Indicator        Value
0  2022  Fertilizer consumption (% of fertilizer produc...  1461.397758
1  2021  Fertilizer consumption (% of fertilizer produc...  1461.397758
2  2020  Fertilizer consumption (% of fertilizer produc...  2838.094220
3  2019  Fertilizer consumption (% of fertilizer produc...  1803.554906
4  2018  Fertilizer consumption (% of fertilizer produc...  1312.520084


In [ ]:
#saved the cleaned df as a csv
df.to_csv('cleaned_indicators_lka.csv', index=False)